In [1]:
import os
import pandas as pd
import numpy as np
import glob

In [9]:
sub = 'sub-04'
ses = 'ses-02'
battery = 'camcan'
task = 'oddball'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = sorted(glob.glob('*' + task + '*.csv'))
tsv_files = []
# for i in range(len(csv_files)):
i = 1
tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [3]:
csv_files

['extracted_oddball_sub-4_run-01_962021-96.csv',
 'extracted_oddball_sub-4_run-02_962021-914.csv']

In [8]:
tsv_files

['oddball_sub-07_run-01.tsv']

In [10]:
file_df = pd.read_csv(os.path.join(datadir, csv_files[i]))

In [11]:
# In the csv file extracted from the E-Prime txt file, 
# run onset is the 'GetReady.OffsetTime' column of last row, 
# and all other columns of this row are empty.
run_start = (file_df.loc[~np.isnan(file_df['GetReady.OffsetTime'])])['GetReady.OffsetTime']
run_start = run_start.values[0]

In [12]:
run_start

59634.0

In [13]:
# Trial onsets are in 'CatImage.OnsetTime'
trial_onsets = (file_df.loc[~np.isnan(file_df['CatImage.OnsetTime'])])['CatImage.OnsetTime']

# Trial offsets are in 'CatImage.OffsetTime'
trial_offsets = (file_df.loc[~np.isnan(file_df['CatImage.OffsetTime'])])['CatImage.OffsetTime']

# In this task, trial duration and RT are the same
durations = trial_offsets - trial_onsets

# Finally, trial_types are in 'Running'
trial_types = (file_df.loc[~np.isnan(file_df['CatImage.OffsetTime'])])['Running']
trial_types[trial_types=='CatEasy'] = 'easy'
trial_types[trial_types=='CatHard'] = 'hard'

/home/sshankar/env/fastsrm/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [14]:
# Responses are in 'CatImage.RESP'. This field is not used in tsv file
# generation but is useful for behavioral analysis.
responses = (file_df.loc[~np.isnan(file_df['CatImage.OnsetTime'])])['CatImage.RESP']

In [9]:
# Quick analysis to look at mean RTs for easy and difficult trials
# easy RTs < difficult RTs

# Find the indices of the easy trials
easy = trial_types == 'CatEasy'

# extract all the RTs
RTs = trial_offsets - trial_onsets

# Calculate mean RTs
easy_mean = np.mean(RTs[easy])
difficult_mean = np.mean(RTs[~easy])

print(easy_mean, difficult_mean)

616.5436893203884 1929.0816326530612


In [14]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start)/1000, 'duration':durations/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)